In [1]:
from lib.Agents import *
from lib.Demand import *
from lib.Env import *

Using TensorFlow backend.


In [ ]:
env = RebalancingEnv(AMoD(5, demand, V=20))

nb_actions = env.action_space.n
input_shape = (1,) + env.state.shape
input_dim = env.input_dim

model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [ ]:
# dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
# anim = draw(env.amods)
# HTML(anim.to_html5_video())

In [7]:
import csv

env = RebalancingEnv(AMoD(5, demand, V=20))

nb_actions = env.action_space.n
input_shape = (1,) + env.state.shape
input_dim = env.input_dim

model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

runtime = []
waittime = []
vehtime = []
servtime = []
rebltime = []

for _ in range(10):
    amod = AMoD(5, demand, V=20)
    dqn.load_weights('dqn_weights.h5f')
    
    stime = time.time()
    amods = []
    for T in range(0,10000,10):
        amod.dispatch_at_time(T)
        if np.remainder(T, 10) == 0:
            amod.rebalance("rl", dqn)
        if np.remainder(T, 100) == 0:
            print("run %d, t = %.0f" % (_, T) )
        amods.append( copy.deepcopy(amod) )
    etime = time.time()    
    rt = etime - stime
    runtime.append(rt)
    
    wt = 0.0
    vt = 0.0
    c_wt = 0
    c_vt = 0
    for r in amod.reqs:
        if r.Tp > -1:
            wt += (r.Tp - r.Tr)
            c_wt += 1
            if r.Td > -1:
                vt += (r.Td - r.Tp)
                c_vt += 1
    wt = wt / c_wt
    vt = vt / c_vt
    waittime.append(wt)
    vehtime.append(vt)
    
    st = 0.0
    rt = 0.0
    for v in amod.vehs:
        st += v.Ts
        rt += v.Tr
    st = st / amod.V
    rt = rt / amod.V
    servtime.append(st)
    rebltime.append(rt)

# f = open('results.csv', 'a');
# writer = csv.writer(f)
# writer.writerow(runtime)
# writer.writerow(waittime)
# writer.writerow(vehtime)
# writer.writerow(servtime)
# writer.writerow(rebltime)
# f.close()

run 0, t = 0
run 0, t = 100
run 0, t = 200
run 0, t = 300
run 0, t = 400
run 0, t = 500
run 0, t = 600
run 0, t = 700
run 0, t = 800
run 0, t = 900
run 0, t = 1000
run 0, t = 1100
run 0, t = 1200
run 0, t = 1300
run 0, t = 1400
run 0, t = 1500
run 0, t = 1600
run 0, t = 1700
run 0, t = 1800
run 0, t = 1900
run 0, t = 2000
run 0, t = 2100
run 0, t = 2200
run 0, t = 2300
run 0, t = 2400
run 0, t = 2500
run 0, t = 2600
run 0, t = 2700
run 0, t = 2800
run 0, t = 2900
run 0, t = 3000
run 0, t = 3100
run 0, t = 3200
run 0, t = 3300
run 0, t = 3400
run 0, t = 3500
run 0, t = 3600
run 0, t = 3700
run 0, t = 3800
run 0, t = 3900
run 0, t = 4000
run 0, t = 4100
run 0, t = 4200
run 0, t = 4300
run 0, t = 4400
run 0, t = 4500
run 0, t = 4600
run 0, t = 4700
run 0, t = 4800
run 0, t = 4900
run 0, t = 5000
run 0, t = 5100
run 0, t = 5200
run 0, t = 5300
run 0, t = 5400
run 0, t = 5500
run 0, t = 5600
run 0, t = 5700
run 0, t = 5800
run 0, t = 5900
run 0, t = 6000
run 0, t = 6100
run 0, t = 6200
run 

In [8]:
np.mean(waittime)

20.82748426331613

In [ ]:
for _ in range(1):
    amod = AMoD(5, demand, V=20)
#     dqn.load_weights('dqn_weights.h5f')
    
    stime = time.time()
    amods = []
    for T in range(0,160,10):
        amod.dispatch_at_time(T)
        if np.remainder(T, 50) == 0:
            amod.rebalance("optimal", dqn)
        if np.remainder(T, 100) == 0:
            print("run %d, t = %.0f" % (_, T) )
        amods.append( copy.deepcopy(amod) )
    etime = time.time()

In [ ]:
def draw_amod(amod):
    plt.figure(figsize=(8,8))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    for v in amod.vehs:
        plt.plot(v.lat, v.lng, 'blue' if v.id == 0 else 'black', marker='o')
        slat = v.lat
        slng = v.lng
        for l in range(len(v.jobs)):
            elat = v.jobs[l][2]
            elng = v.jobs[l][3]
            plt.plot([slat, elat], [slng, elng],
                     'red' if v.is_rebalancing() else 'black',
                     linestyle=':')
            slat = elat
            slng = elng
#     for l in [0.2, 0.4, 0.6, 0.8]:
#         plt.plot([0, 1], [l, l], 'grey', linestyle='-', linewidth=0.5)
#         plt.plot([l, l], [0, 1], 'grey', linestyle='-', linewidth=0.5)
    lat, lng = amod.vehs[0].get_location()  
    for l in [-0.25, -0.15, -0.05, 0.05, 0.15, 0.25]:
        plt.plot([lat+l, lat+l], [lng-0.25, lng+0.25], 'grey', linestyle='-', linewidth=0.5)
        plt.plot([lat-0.25, lat+0.25], [lng+l, lng+l], 'grey', linestyle='-', linewidth=0.5)
    plt.show()
    
draw_amod(amod)

In [ ]:
fig1 = plt.figure();

ind = np.arange(1,9);

ax1 = fig1.add_subplot(131);
ax1.boxplot(results[0][0].transpose());
# ax1.plot(ind, np.median(results[0][0], axis=1), color='r', ls='--');
ax1.set_title('Non-shared Vehicle');
ax1.set_ylim([0, 1.05]);
ax1.set_xlabel('Number of Vehicles');
ax1.set_ylabel('Request Response Rate');

ax2 = fig1.add_subplot(132);
ax2.boxplot(results[0][1].transpose());
# ax2.plot(ind, np.median(results[0][1], axis=1), color='r', ls='--');
ax2.set_title('2-seat Shared Vehicle');
ax2.set_ylim([0, 1.05]);
ax2.set_xlabel('Number of Vehicles');

ax3 = fig1.add_subplot(133);
ax3.boxplot(results[0][2].transpose());
# ax3.plot(ind, np.median(results[0][2], axis=1), color='r', ls='--');
ax3.set_title('4-seat Shared Vehicle');
ax3.set_ylim([0, 1.05]);
ax3.set_xlabel('Number of Vehicles');


fig2 = plt.figure();

ind = np.arange(1,9);
width = 0.4;

ax1 = fig2.add_subplot(131);
bar1 = np.mean(results[1][0], axis=1);
bar2 = np.mean(results[2][0], axis=1);
bar3 = np.mean(results[3][0], axis=1);
err1 = np.std(results[1][0], axis=1);
err2 = np.std(results[2][0], axis=1);
err3 = np.std(results[3][0], axis=1);
p1 = ax1.bar(ind-width/2, bar1, width, color='cyan', yerr=err1);
p2 = ax1.bar(ind-width/2, bar2, width, bottom=bar1, color='yellow', yerr=err2);
p3 = ax1.bar(ind-width/2, bar3, width, bottom=bar1+bar2, color='orange', yerr=err3);
ax1.set_ylim([0, 1400]);
ax1.set_title('Non-shared Vehicle');
ax1.set_xlabel('Number of Vehicles');
ax1.set_ylabel('Average Response/Wait/Travel Time (s)');

ax2 = fig2.add_subplot(132);
bar1 = np.mean(results[1][1], axis=1);
bar2 = np.mean(results[2][1], axis=1);
bar3 = np.mean(results[3][1], axis=1);
err1 = np.std(results[1][1], axis=1);
err2 = np.std(results[2][1], axis=1);
err3 = np.std(results[3][1], axis=1);
p1 = ax2.bar(ind-width/2, bar1, width, color='cyan', yerr=err1);
p2 = ax2.bar(ind-width/2, bar2, width, bottom=bar1, color='yellow', yerr=err2);
p3 = ax2.bar(ind-width/2, bar3, width, bottom=bar1+bar2, color='orange', yerr=err3);
ax2.set_ylim([0, 1400]);
ax2.set_title('2-seat Shared Vehicle');
ax2.set_xlabel('Number of Vehicles');

ax3 = fig2.add_subplot(133);
bar1 = np.mean(results[1][2], axis=1);
bar2 = np.mean(results[2][2], axis=1);
bar3 = np.mean(results[3][2], axis=1);
err1 = np.std(results[1][2], axis=1);
err2 = np.std(results[2][2], axis=1);
err3 = np.std(results[3][2], axis=1);
p1 = ax3.bar(ind-width/2, bar1, width, color='cyan', yerr=err1);
p2 = ax3.bar(ind-width/2, bar2, width, bottom=bar1, color='yellow', yerr=err2);
p3 = ax3.bar(ind-width/2, bar3, width, bottom=bar1+bar2, color='orange', yerr=err3);
ax3.set_ylim([0, 1400]);
ax3.set_title('4-seat Shared Vehicle');
ax3.set_xlabel('Number of Vehicles');

plt.show();

In [ ]:
anim = draw(amods)
HTML(anim.to_html5_video())

In [ ]:
anim.save('rl.mp4', fps=30, extra_args=['-vcodec', 'libx264'])